In [2]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# Set up Selenium options and driver
options = Options()
options.add_argument("--disable-gpu")
# options.add_argument("--headless")  # Optional: Run in headless mode

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Navigate to the page
base_url = "https://www.nykaa.com/brands/the-ordinary/c/23993?"
params = "intcmp=brand_menu,luxe,The%20Ordinary&redirectpath=slug"
paginationNumber = 1

# Allow initial time for the page to load
time.sleep(5)

# Scroll incrementally until all products are loaded
scroll_pause_time = 2  # Wait time after each scroll

# Lists to store data
productLinks = []
products = []

while True:

    if paginationNumber == 1:
        url = f"{base_url}{params}"
    else:
        url = f"{base_url}page_no={paginationNumber}&sort=popularity&intcmp=brand_menu,luxe,The%20Ordinary&redirectpath=slug&eq=desktop"
    
    # Load the Page
    driver.get(url)
    print(f"Scraping Page {paginationNumber}: {url}")
    time.sleep(3)
    
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

    # Wait until the products are loaded
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.productWrapper.css-17nge1h'))
        )
    except TimeoutException:
        print(f"Scraped the Total number of pages: {paginationNumber - 1}")
        break

    # Parse the page content with BeautifulSoup
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')
    
    # Find all product items
    deygaOrganicProducts = soup.findAll('div', class_='productWrapper css-17nge1h')
    if not deygaOrganicProducts:
        print("No Products found on this page.")
        break

    # Extract product links
    for product in deygaOrganicProducts:
        # Extract the Stock Status
        stockTag = product.find('span', class_ = "css-lg5xc9")
        if stockTag:
            stock = stockTag.get_text(strip = True)
        else:
            stock = "In Stock"

        # Extract the Product Highlight
        highlightTag = product.find("li", class_ = "custom-tag css-1bse542")
        if highlightTag:
            highlight = highlightTag.get_text(strip = True)
        else:
            highlight = "None"

        # Extract product link
        linkTag = product.find('a', href=True)
        if linkTag:
            productLink = linkTag['href']
            # Check if the link is a relative URL and add the base URL if necessary
            if not productLink.startswith("http"):  # Relative URL
                productLink = "https://www.nykaa.com" + productLink
            productLinks.append({"link" : productLink, "stock" : stock, "highlight" : highlight})
                
    for productData in productLinks:
        
        link = productData["link"]
        stock = productData["stock"]
        highlight = productData["highlight"]
        
        driver.get(link)
        time.sleep(3)  # Allow time for the page to load

        # Parse individual product page
        productPageContent = driver.page_source
        productSoup = BeautifulSoup(productPageContent, 'html.parser')
    
        # Extract Product Name
        productTag = productSoup.find('h1', class_='css-1gc4x7i')
        if productTag:
            prodName = productTag.get_text(strip=True)
        else:
            prodName = "No Description Available"
    
        # Extract Ratings
        ratingTag = productSoup.find('div', class_="css-1m0y15j")
        if ratingTag:
            ratingText = ratingTag.find('div', class_="css-m6n3ou")
            if ratingText:
                ratings = ratingText.get_text(strip = True)
            else:
                ratings = "N/A"
        else:
            ratings = "N/A"

        # Extract the Ratings & Reviews Count
        ratingCountTag = productSoup.find('div', class_ = "css-1eip5u4")
        if ratingCountTag:
            ratingCount = ratingCountTag.get_text(strip = True)
        else:
            ratingCount = "None"

        # Extract the Offer Price and Original Price
        productDiv = productSoup.find('div', class_ = "css-1d0jf8e")

        # Extract the first two (span) tags
        if productDiv:
            spanTags = productDiv.find_all('span')
            # print(len(spanTags))

            originalPrice = "N/A"
            offerPrice = "N/A"

            if len(spanTags) == 2:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = "N/A"
            elif len(spanTags) == 4:
                originalPrice = spanTags[1].get_text(strip = True)
                offerPrice = spanTags[2].get_text(strip = True)

            # Print the Values
            # print(f"Original Price: {originalPrice}, Offer Price: {offerPrice}")

        else:
            print("Product div not found, defaulting to Unavailable for prices")

        # Extract the Discount
        discountTag = productSoup.find('span', class_ = "css-bhhehx")
        if discountTag:
            discount = discountTag.get_text(strip = True)
        else:
            discount = "No Discount"

        products.append({
            "Product Brand" : "The Ordinary.",
            "Product Name" : prodName,
            "Product Rating" : ratings,
            "Product Rating & Review Count" : ratingCount,
            "Product Original Price" : originalPrice, 
            "Product Offer Price" : offerPrice, 
            "Product Discount" : discount,
            "Product Highlight" : highlight,
            "Product Stock Status" : stock
        })

    # Clear the product links list to avoid the duplication
    productLinks.clear()

    paginationNumber += 1 # Increment the Page Number
    time.sleep(5)

# Close the browser after scraping
driver.quit()

Scraping Page 1: https://www.nykaa.com/brands/the-ordinary/c/23993?intcmp=brand_menu,luxe,The%20Ordinary&redirectpath=slug
Scraping Page 2: https://www.nykaa.com/brands/the-ordinary/c/23993?page_no=2&sort=popularity&intcmp=brand_menu,luxe,The%20Ordinary&redirectpath=slug&eq=desktop
Scraping Page 3: https://www.nykaa.com/brands/the-ordinary/c/23993?page_no=3&sort=popularity&intcmp=brand_menu,luxe,The%20Ordinary&redirectpath=slug&eq=desktop
Scraping Page 4: https://www.nykaa.com/brands/the-ordinary/c/23993?page_no=4&sort=popularity&intcmp=brand_menu,luxe,The%20Ordinary&redirectpath=slug&eq=desktop
Scraped the Total number of pages: 3


In [4]:
print(f"Products: \n {products}")

Products: 
 [{'Product Brand': 'The Ordinary.', 'Product Name': 'The Ordinary Niacinamide 10% + Zinc 1%(30ml)', 'Product Rating': '4.4/5', 'Product Rating & Review Count': '73496ratings&5498reviews', 'Product Original Price': '₹600', 'Product Offer Price': 'N/A', 'Product Discount': 'No Discount', 'Product Highlight': 'None', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'The Ordinary.', 'Product Name': 'The Ordinary GF 15% Solution(30ml)', 'Product Rating': '4.9/5', 'Product Rating & Review Count': '7ratings&2reviews', 'Product Original Price': '₹1550', 'Product Offer Price': 'N/A', 'Product Discount': 'No Discount', 'Product Highlight': 'None', 'Product Stock Status': 'In Stock'}, {'Product Brand': 'The Ordinary.', 'Product Name': 'The Ordinary Glycolic Acid 7% Exfoliating Mini Toner(100ml)', 'Product Rating': '4.4/5', 'Product Rating & Review Count': '31051ratings&2750reviews', 'Product Original Price': '₹900', 'Product Offer Price': 'N/A', 'Product Discount': 'No Discount'

In [5]:
theOrdinary = pd.DataFrame(products)
theOrdinary

,Product Brand,Product Name,Product Rating,Product Rating & Review Count,Product Original Price,Product Offer Price,Product Discount,Product Highlight,Product Stock Status
0,The Ordinary.,The Ordinary Niacinamide 10% + Zinc 1%(30ml),4.4/5,73496ratings&5498reviews,₹600,N/A,No Discount,None,In Stock
1,The Ordinary.,The Ordinary GF 15% Solution(30ml),4.9/5,7ratings&2reviews,₹1550,N/A,No Discount,None,In Stock
2,The Ordinary.,The Ordinary Glycolic Acid 7% Exfoliating Mini...,4.4/5,31051ratings&2750reviews,₹900,N/A,No Discount,None,In Stock
3,The Ordinary.,The Ordinary Multi-Peptide Hair Serum For Dens...,4.4/5,13742ratings&1414reviews,₹2150,N/A,No Discount,BESTSELLER,In Stock
4,The Ordinary.,The Ordinary AHA 30% + BHA 2% Peeling Solution...,4.4/5,21214ratings&2112reviews,₹950,N/A,No Discount,BESTSELLER,In Stock
5,The Ordinary.,The Ordinary Mini Discovery Set(225ml),4.2/5,10ratings,₹2850,N/A,No Discount,None,In Stock
6,The Ordinary.,The Ordinary Caffeine Solution 5% + EGCG(30ml),4.3/5,7776ratings&738reviews,₹900,N/A,No Discount,None,In Stock
7,The Ordinary.,The Ordinary Saccharomyces Ferment 30% Milky T...,4.9/5,19ratings&3reviews,₹1500,N/A,No Discount,None,In Stock
8,The Ordinary.,The Ordinary Multi-Peptide + Copper Peptides 1...,4.5/5,910ratings&88reviews,₹2950,N/A,No Discount,BESTSELLER,In Stock
9,The Ordinary.,The Ordinary Salicylic Acid 0.5% Body Serum(24...,4.7/5,10ratings&2reviews,₹1500,N/A,No Discount,None,In Stock


In [6]:
# theOrdinary.to_csv("C:/Users/Logiya Vidhyapathy/Documents/KGISL Data Science/Project Files and Documents/Capstone Project/The Ordinary. Products_Nykaa.csv", index = False)